In [4]:
import urllib.request
import pandas as pd
from datetime import datetime
import os
import time
import re
import tabulate
print("Setup complete!") 

Setup complete!


In [8]:
def download_csv(country, year_1, year_2, type_data, directory):
    for province_ID in range(1, 28):
        url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country={country}&provinceID={province_ID}&year1={year_1}&year2={year_2}&type={type_data}"
       
        retries = 3
        for attempt in range(retries):
            try:
                with urllib.request.urlopen(url) as wp:
                    text = wp.read()
                break
            except urllib.error.URLError as e:
                print(f"Error downloading data for ID {province_ID}: {e}")
                if attempt < retries - 1:
                    print(f"Retrying download for ID {province_ID} in 5 seconds...")
                    time.sleep(5)
                else:
                    print(f"Failed to download data for ID {province_ID} after {retries} attempts.")
                    continue


        current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        filename = f'NOAA_ID_{province_ID}_{current_time}.csv'
        filepath = os.path.join(directory, filename)


        if not os.path.exists(filepath):
            try:
                with open(filepath, 'wb') as out:
                    out.write(text)
                print(f"File_csv:{filename} downloaded successfully.")
            except IOError as e:
                print(f"Error writing file {filename}: {e}")
        else:
            print(f"File already exists: {filename}, skipping download.")


def clean_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"[+] Directory created successfully.")
   
    if input("Do you want to clean the directory? (y/n): ").lower() == 'y':
        for filename in os.listdir(directory):
            filepath = os.path.join(directory, filename)
            if os.path.exists(filepath):
                os.remove(filepath)
        print(f"[+] Directory cleaned successfully.\n")                
    else:
        print(f"[+] Directory cleaning skipped.\n")



country = "UKR"
year_1 = 1981
year_2 = 2025
type_data = "Mean"
directory = "C:/Users/Alina/ad_project/venv/data_csv"  


clean_directory(directory)    
download_csv(country, year_1, year_2, type_data, directory)




Do you want to clean the directory? (y/n):  y


[+] Directory cleaned successfully.

File_csv:NOAA_ID_1_2025-03-20_02-10-39.csv downloaded successfully.
File_csv:NOAA_ID_2_2025-03-20_02-10-41.csv downloaded successfully.
File_csv:NOAA_ID_3_2025-03-20_02-10-42.csv downloaded successfully.
File_csv:NOAA_ID_4_2025-03-20_02-10-44.csv downloaded successfully.
File_csv:NOAA_ID_5_2025-03-20_02-10-45.csv downloaded successfully.
File_csv:NOAA_ID_6_2025-03-20_02-10-47.csv downloaded successfully.
File_csv:NOAA_ID_7_2025-03-20_02-10-48.csv downloaded successfully.
File_csv:NOAA_ID_8_2025-03-20_02-10-50.csv downloaded successfully.
File_csv:NOAA_ID_9_2025-03-20_02-10-51.csv downloaded successfully.
File_csv:NOAA_ID_10_2025-03-20_02-10-53.csv downloaded successfully.
File_csv:NOAA_ID_11_2025-03-20_02-10-55.csv downloaded successfully.
File_csv:NOAA_ID_12_2025-03-20_02-10-56.csv downloaded successfully.
File_csv:NOAA_ID_13_2025-03-20_02-10-58.csv downloaded successfully.
File_csv:NOAA_ID_14_2025-03-20_02-11-00.csv downloaded successfully.
File_c

In [9]:
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def read_csv_file(filepath):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    try:
        df = pd.read_csv(filepath, header=1, names=headers, converters={'Year': remove_html_tags})
        df = df.drop(df.loc[df['VHI'] == -1].index)
        df = df.drop("empty", axis=1)
        df = df.drop(df.index[-1])          
        return df
    
    except pd.errors.ParserError as e:
        print(f"Error reading {filepath}: {e}")
        return None

def read_data(directory):
    if not os.path.isdir(directory):
        print(f"The directory {directory} does not exist or is not accessible.")
        return None
    
    data_frames = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if filepath.endswith('.csv'):
            province_ID = filename.split('_')[2] if len(filename.split('_')) >= 3 else None                          
            
            df = read_csv_file(filepath)  
            if province_ID and int(province_ID) not in [12, 20]:          
                if df is not None:
                    df.insert(0, "PROVINCE_ID", int(province_ID), True)                   
                    df["Week"] = df["Week"].astype(int)
                    df["Year"] = df["Year"].astype(int)               
                    data_frames.append(df)                 
    
    if data_frames:
        data_frames.sort(key=lambda x: int(x["PROVINCE_ID"].iloc[0]))
        data_frames = pd.concat(data_frames).drop_duplicates().reset_index(drop=True)                      
        return data_frames
    else:
        print("No valid CSV files found.")
        return None

def print_dataframe(df):
    if df is not None:
        print(tabulate.tabulate(df, headers='keys', tablefmt="heavy_grid", showindex=False))
    else:
        print("No data to display.")

In [10]:
directory = "C:/Users/Alina/ad_project/venv/data_csv"
data_frames_1 = read_data(directory)
print_dataframe(data_frames_1.head(10))
print_dataframe(data_frames_1.tail(10)) 

┏━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   PROVINCE_ID ┃   Year ┃   Week ┃   SMN ┃    SMT ┃   VCI ┃   TCI ┃   VHI ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃             1 ┃   1982 ┃      1 ┃ 0.053 ┃ 260.31 ┃ 45.01 ┃ 39.46 ┃ 42.23 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃             1 ┃   1982 ┃      2 ┃ 0.054 ┃ 262.29 ┃ 46.83 ┃ 31.75 ┃ 39.29 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃             1 ┃   1982 ┃      3 ┃ 0.055 ┃ 263.82 ┃ 48.13 ┃ 27.24 ┃ 37.68 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃             1 ┃   1982 ┃      4 ┃ 0.053 ┃ 265.33 ┃ 46.09 ┃ 23.91 ┃ 35    ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃             1 ┃   1982 ┃      5 ┃ 0.05  ┃ 265.66 ┃ 41.46 ┃ 26.65 ┃ 34.06 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫

In [11]:
def searching(directory):
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)
        if file_path.endswith('.csv'):
            search_id_city(file_path)

def search_id_city(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
        search_results = re.findall(r"= (\d+): ([\w'' '-]+),", content)     
        print(search_results) 
        return search_results


In [12]:
directory = "C:/Users/Alina/ad_project/venv/data_csv"
searching(directory)

[('10', "Khmel'nyts'kyy")]
[('11', 'Kiev')]
[('12', 'Kiev City')]
[('13', 'Kirovohrad')]
[('14', "Luhans'k")]
[('15', "L'viv")]
[('16', 'Mykolayiv')]
[('17', 'Odessa')]
[('18', 'Poltava')]
[('19', 'Rivne')]
[('1', 'Cherkasy')]
[('20', "Sevastopol'")]
[('21', 'Sumy')]
[('22', "Ternopil'")]
[('23', 'Transcarpathia')]
[('24', 'Vinnytsya')]
[('25', 'Volyn')]
[('26', 'Zaporizhzhya')]
[('27', 'Zhytomyr')]
[('2', 'Chernihiv')]
[('3', 'Chernivtsi')]
[('4', 'Crimea')]
[('5', "Dnipropetrovs'k")]
[('6', "Donets'k")]
[('7', "Ivano-Frankivs'k")]
[('8', 'Kharkiv')]
[('9', 'Kherson')]


In [17]:
def replace_function(data_frames):
    
    province_ID_dictionary = {
        16: 13, 27: 5, 10: 21, 3: 23, 11: 9, 25: 2, 17: 14, 21: 17, 1: 22,
        18: 15, 6: 4, 9: 20, 8: 19, 19: 16, 23: 6, 7: 8, 5: 3, 26: 7, 2: 24,
        4: 25, 15: 11, 22: 18, 13: 10, 24: 1, 14: 12, 12: 27, 20: 26 
    }

    if "PROVINCE_ID" not in data_frames.columns:
        print("no column 'PROVINCE_ID' in DataFrame.")
        return data_frames  

    data_frames_cp = data_frames.copy()
    data_frames_cp["PROVINCE_ID"] = data_frames_cp["PROVINCE_ID"].replace(province_ID_dictionary)
    
    print(data_frames_cp["PROVINCE_ID"].unique())

    print("Indexes updated")

    return data_frames_cp



In [18]:
directory = "C:/Users/Alina/ad_project/venv/data_csv"
data_frames = read_data(directory)        
data_frames_replace = replace_function(data_frames_1) 
print_dataframe(data_frames_replace.head(5))
print_dataframe(data_frames_replace.tail(5))

[22 24 23 25  3  4  8 19 20 21  9 10 12 11 13 14 15 16 17 18  6  1  2  7
  5]
Indexes updated
┏━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   PROVINCE_ID ┃   Year ┃   Week ┃   SMN ┃    SMT ┃   VCI ┃   TCI ┃   VHI ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃            22 ┃   1982 ┃      1 ┃ 0.053 ┃ 260.31 ┃ 45.01 ┃ 39.46 ┃ 42.23 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃            22 ┃   1982 ┃      2 ┃ 0.054 ┃ 262.29 ┃ 46.83 ┃ 31.75 ┃ 39.29 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃            22 ┃   1982 ┃      3 ┃ 0.055 ┃ 263.82 ┃ 48.13 ┃ 27.24 ┃ 37.68 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃            22 ┃   1982 ┃      4 ┃ 0.053 ┃ 265.33 ┃ 46.09 ┃ 23.91 ┃ 35    ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃            22 ┃   1982 ┃      5 ┃ 0.05  ┃ 265.66 ┃ 41.46 

In [32]:
def VHI_extremes(province_ID, year_1, year_2, df):
    PROVINCE_NAME_dict = {1: 'Вінницька',  2: 'Волинська',  3: 'Дніпропетровська',  4: 'Донецька',  5: 'Житомирська',
        6: 'Закарпатська',  7: 'Запорізька',  8: 'Івано-Франківська',  9: 'Київська',  10: 'Кіровоградська',
        11: 'Луганська',  12: 'Львівська',  13: 'Миколаївська',  14: 'Одеська',  15: 'Полтавська',
        16: 'Рівенська',  17: 'Сумська',  18: 'Тернопільська',  19: 'Харківська',  20: 'Херсонська',
        21: 'Хмельницька',  22: 'Черкаська',  23: 'Чернівецька',  24: 'Чернігівська',  25: 'Республіка Крим' }
    
    PROVINCE_NAME = PROVINCE_NAME_dict[province_ID]
    
    filtered_df = df[(df["PROVINCE_ID"] == province_ID) & (df["Year"] >= year_1) & (df["Year"] <= year_2)]
    vhi_values = filtered_df['VHI']

    vhi_values_filtered = vhi_values[vhi_values != -1]

    if not vhi_values_filtered.empty:
        min_value = vhi_values_filtered.min()
        max_value = vhi_values_filtered.max()
        mean_value = vhi_values_filtered.mean()
        median_value = np.median(vhi_values_filtered)
    else:
        min_value = max_value = mean_value = median_value = None
    
    return [province_ID, PROVINCE_NAME, min_value, max_value, mean_value, median_value]


def VHI_main():
    directory = "C:/Users/Alina/ad_project/venv/data_csv"
    data_frames_1 = read_data(directory)
    data_frames_replace = replace_function(data_frames_1)  
    
    year_1 = 2025
    year_2 = 2025
    
    vhi_extremes_data = []
    headers = ["Province ID", "PROVINCE_NAME", "Min VHI", "Max VHI", "Mean VHI", "Median VHI"]

    for province_ID in range(1, 26):  
        vhi_extremes_data.append(VHI_extremes(province_ID, year_1, year_2, data_frames_replace)) 

    print(tabulate.tabulate(vhi_extremes_data, headers=headers, tablefmt="github"))





In [33]:
VHI_main()

[22 24 23 25  3  4  8 19 20 21  9 10 12 11 13 14 15 16 17 18  6  1  2  7
  5]
Indexes updated
|   Province ID | PROVINCE_NAME     |   Min VHI |   Max VHI |   Mean VHI |   Median VHI |
|---------------|-------------------|-----------|-----------|------------|--------------|
|             1 | Вінницька         |     40.08 |     44.2  |    42.1036 |        41.61 |
|             2 | Волинська         |     44.41 |     49.46 |    47.2973 |        47.28 |
|             3 | Дніпропетровська  |     32.27 |     38.7  |    36.2291 |        36.37 |
|             4 | Донецька          |     29.47 |     39.54 |    35.3218 |        36.14 |
|             5 | Житомирська       |     38.55 |     46.11 |    43.4155 |        44.52 |
|             6 | Закарпатська      |     39.13 |     60.12 |    47.9873 |        48.08 |
|             7 | Запорізька        |     34.17 |     39.74 |    37.2791 |        37.15 |
|             8 | Івано-Франківська |     44.06 |     51    |    46.1109 |        45.48 |
|     

In [61]:
def check_extreme_and_moderate(df, threshold, VHI_range):

    filtered_data = df[(df['VHI'] >= VHI_range[0]) & (df['VHI'] <= VHI_range[1])]

    years_grouped = filtered_data.groupby('Year').filter(lambda x: len(x) / len(df['PROVINCE_ID'].unique()) > threshold)
    
    return years_grouped

def VHI_main_2():
    # Ініціалізація словника провінцій
    PROVINCE_NAME_dict = {
        1: 'Вінницька',  2: 'Волинська',  3: 'Дніпропетровська',  4: 'Донецька',  5: 'Житомирська',
        6: 'Закарпатська',  7: 'Запорізька',  8: 'Івано-Франківська',  9: 'Київська',  10: 'Кіровоградська',
        11: 'Луганська',  12: 'Львівська',  13: 'Миколаївська',  14: 'Одеська',  15: 'Полтавська',
        16: 'Рівенська',  17: 'Сумська',  18: 'Тернопільська',  19: 'Харківська',  20: 'Херсонська',
        21: 'Хмельницька',  22: 'Черкаська',  23: 'Чернівецька',  24: 'Чернігівська',  25: 'Республіка Крим'
    }
    
    directory =  "C:/Users/Alina/ad_project/venv/data_csv"
    data_frames_1 = read_data(directory)
    data_frames_replace = replace_function(data_frames_1)

    year_1 = 1982
    year_2 = 2024  

    vhi_extremes_data = []
    headers = ["Province ID", "PROVINCE_NAME", "Year_Start", "Year_End", "VHI"]

    for province_ID in range(1, 11):  
        province_vhi_data = VHI_extremes(province_ID, year_1, year_2, data_frames_replace)

        vhi_display = str(province_vhi_data[:10]) + '...' if len(province_vhi_data) > 10 else str(province_vhi_data)
        
        vhi_extremes_data.append([province_ID, PROVINCE_NAME_dict[province_ID], year_1, year_2, vhi_display])

    print(tabulate.tabulate(vhi_extremes_data, headers=headers, tablefmt="grid"))

    VHI = [0, 15]
    extreme_years = check_extreme_and_moderate(data_frames_replace, 0.2, VHI)
    print(f"=" * 155)
    print(f"Extreme drought years affecting more than 20% of provinces:")
    print(extreme_years[['Year', 'PROVINCE_ID', 'VHI']].head()) 
    print(f"=" * 155)
        
    VHI = [15, 35]    
    moderate_years = check_extreme_and_moderate(data_frames_replace, 0.2, VHI)  
    print(f"Moderate drought years affecting more than 20% of provinces:")
    print(moderate_years[['Year', 'PROVINCE_ID', 'VHI']].head()) 
    print(f"=" * 155)


In [62]:
VHI_main_2()


[22 24 23 25  3  4  8 19 20 21  9 10 12 11 13 14 15 16 17 18  6  1  2  7
  5]
Indexes updated
+---------------+-------------------+--------------+------------+---------------------------------------------------------------------------+
|   Province ID | PROVINCE_NAME     |   Year_Start |   Year_End | VHI                                                                       |
+===============+===================+==============+============+===========================================================================+
|             1 | Вінницька         |         1982 |       2024 | [45.9, 45.34, 44.88, 41.6, 39.29, 37.65, 35.03, 34.46, 35.04, 34.33]...   |
+---------------+-------------------+--------------+------------+---------------------------------------------------------------------------+
|             2 | Волинська         |         1982 |       2024 | [53.88, 50.97, 45.65, 40.04, 36.44, 35.14, 33.57, 33.23, 33.53, 35.07]... |
+---------------+-------------------+--------------+--